In [ ]:
import io
import math
import os
import pathlib

import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import panel as pn
import pandas as pd
import pymap3d
import xarray as xr

hv.extension("bokeh")
np.set_printoptions(suppress=True)

In [2]:
def get_skews_and_base_cfls(lons, lats, depths) -> np.ndarray:
    # The shape of each one of the input arrays needs to be (3, <no_triangles>)
    #ell = pymap3d.Ellipsoid.from_name("wgs84")
    ell = pymap3d.Ellipsoid(6378206.4, 6378206.4, "schism", "schism")
    local_x, local_y, _ = pymap3d.geodetic2enu(lats, lons, depths, lats[0], lons[0], depths[0], ell=ell)
    areas = (local_x[1] * local_y[2] - local_x[2] * local_y[1]) * 0.5
    rhos = np.sqrt(areas / np.pi)
    max_sides = np.maximum(
        np.sqrt(local_x[1] ** 2 + local_y[1] ** 2),
        np.sqrt(local_x[2] ** 2 + local_y[2] ** 2),
        np.sqrt((local_x[2] - local_x[1]) ** 2 + (local_y[2] - local_y[1]) ** 2),
    )
    skews = max_sides / rhos
    base_cfls = np.sqrt(9.81 * np.maximum(0.1, depths.mean(axis=0))) / rhos / 2
    return skews, base_cfls

def get_skews_and_base_cfls_from_path(path: os.PathLike[str] | str) -> np.ndarray:
    ds = xr.open_dataset(path, engine='selafin')
    tri = ds.attrs['ikle2'] - 1
    lons = ds.x.values[tri].T
    lats = ds.y.values[tri].T
    depths = - ds.B.isel(time=0).values[tri].T
    skews, base_cfls = get_skews_and_base_cfls(lons=lons, lats=lats, depths=depths)
    return skews, base_cfls

In [ ]:
files = {"v1.2" : ["/home/tomsail/Documents/work/models/meshes/slf/v1.2.slf", 50],
         "v2.2" : ["/home/tomsail/Documents/work/models/meshes/slf/v2.2.slf", 30],
         "v1.5" : ["/home/tomsail/Documents/work/models/meshes/slf/v1.5.slf", 50],
         "v2.3" : ["/home/tomsail/Documents/work/models/meshes/slf/v2.3.slf", 30],
         "v3.1" : ["/home/tomsail/Documents/work/models/meshes/slf/v3.1.slf", 20]}

plots = []
for v in files.keys():
    file = files[v][0]
    ideal_dt = files[v][1]
    ds = xr.open_dataset(file, engine='selafin')
    skews, base_cfls = get_skews_and_base_cfls_from_path(file)
    CFL_THRESHOLD = 1
    print(v)
    for dt in (1, 10, 20, 30, 50, 75, 100, 120, 150, 200, 300, 400):
        violations = (base_cfls * dt > CFL_THRESHOLD).sum()
        print(f"{dt:>4d} {violations:>12d} {violations / len(base_cfls) * 100:>8.2f}%")
    _skews = pd.DataFrame({"skew": skews}).hvplot.hist(bins=40,bin_range=[2.5,3.5]).opts(title = v + " skewness")
    _cfls = pd.DataFrame({"cfls": base_cfls * ideal_dt}).hvplot.hist(bins=40, bin_range = [0,1.5]).opts(title = v + " CFL")
    both = (_skews + _cfls)
    plots.append(both)

In [ ]:
hv.Layout(plots).cols(2)

In [ ]:
tri = ds.attrs['ikle2'] - 1
nodes = pd.DataFrame(np.vstack((ds.x, ds.y, ds.B.isel(time=0))).T, columns=["lon", "lat", "depth"])
elements = pd.DataFrame(np.vstack( (np.ones(len(tri))* 3, tri.T)).T , columns=["no_nodes", "n1", "n2", "n3"])
elements = elements.assign(base_cfl=base_cfls)
elements.head()

In [ ]:
min_cfl_per_node = pd.concat([
    elements[["n1", "base_cfl"]].groupby(["n1"]).base_cfl.min(),
    elements[["n2", "base_cfl"]].groupby(["n2"]).base_cfl.min(),
    elements[["n3", "base_cfl"]].groupby(["n3"]).base_cfl.min(),
], axis=1).min(axis=1)
min_cfl_per_node.head()

In [ ]:
dt = 30
df = nodes.assign(
    cfl=min_cfl_per_node * dt,
    # CFL_violation nodes have a value of 1 if there is no violation and 4 if there is a violation. 
    # We do this in order to plot the points with a different size
    cfl_violation=((min_cfl_per_node * dt > CFL_THRESHOLD) * 3) + 1   
)
df.head()

In [ ]:
plot = df[df.cfl_violation == 4].hvplot.points(
    'lon', 
    'lat',
    c="depth",
    cmap="jet",
    geo=True,
    tiles="EsriImagery",
).options(
    width=1200, height=900
)
len(df[df.cfl_violation == 4])

In [ ]:
plot

In [ ]:
plots = []
for v in files.keys():
    file = files[v][0]
    ideal_dt = files[v][1]
    ds = xr.open_dataset(file, engine='selafin')
    skews, base_cfls = get_skews_and_base_cfls_from_path(file)
    CFL_THRESHOLD = 1
    tri = ds.attrs['ikle2'] - 1
    # 
    nodes = pd.DataFrame(np.vstack((ds.x, ds.y, ds.B.isel(time=0))).T, columns=["lon", "lat", "depth"])
    elements = pd.DataFrame(np.vstack( (np.ones(len(tri))* 3, tri.T)).T , columns=["no_nodes", "n1", "n2", "n3"])
    elements = elements.assign(cfls=base_cfls*ideal_dt, skews = skews)
    min_cfl_per_node = pd.concat([
        elements[["n1", "cfls"]].groupby(["n1"]).cfls.min(),
        elements[["n2", "cfls"]].groupby(["n2"]).cfls.min(),
        elements[["n3", "cfls"]].groupby(["n3"]).cfls.min(),
    ], axis=1).min(axis=1)
    mean_skew_per_node = pd.concat([
        elements[["n1", "skews"]].groupby(["n1"]).skews.mean(),
        elements[["n2", "skews"]].groupby(["n2"]).skews.mean(),
        elements[["n3", "skews"]].groupby(["n3"]).skews.mean(),
    ], axis=1).mean(axis=1)
    df = nodes.assign(
        cfl=min_cfl_per_node,
        # CFL_violation nodes have a value of 1 if there is no violation and 4 if there is a violation. 
        # We do this in order to plot the points with a different size
        cfl_violation=((min_cfl_per_node > CFL_THRESHOLD) * 3) + 1  
    )
    plot = df[df.cfl_violation == 4].hvplot.points(
        'lon', 
        'lat',
        c="depth",
        cmap="jet",
        geo=True,
        tiles="EsriImagery",
    ).options(
        width=1200, height=900, title = v
    )
    print(v, len(df[df.cfl_violation == 4]))
    plots.append(plot)

In [ ]:
hv.Layout(plots).cols(1)